# Chapter 5: Your First Agent - Tool Calling

**What you'll learn:**
- How **tools** bridge the gap between LLM reasoning and real-world data
- The **@tool decorator** that transforms Python functions into agent capabilities
- How **CodeAgent** combines an LLM with tools to answer questions autonomously
- How **memory** enables multi-turn conversations

---

**This notebook uses LIVE market data via `yfinance`.** Every price, market cap, and 52-week high comes from Yahoo Finance in real time. No hardcoded dictionaries. The numbers you see will reflect actual market conditions when you run the cells.

```
┌──────────────────────────────────────────────────┐
│                    THE BRIDGE                     │
│                                                   │
│   USER ──→ LLM ──→ TOOL ──→ REAL DATA ──→ LLM   │
│                                                   │
│   "What's AAPL?"  "I should    yfinance API       │
│                    call the    returns live       │
│                    price tool"  market price      │
└──────────────────────────────────────────────────┘
```

In [1]:
!pip install smolagents yfinance -q

^C


In [2]:
from smolagents import CodeAgent, tool
from smolagents import OpenAIServerModel
from smolagents.monitoring import LogLevel

In [3]:
import getpass
API_KEY = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [4]:
model = OpenAIServerModel("gpt-4o-mini", api_key=API_KEY)
print("Model initialized!")

Model initialized!


## The @tool Decorator

The `@tool` decorator transforms a regular Python function into something an LLM can discover and call. It does three things:

1. **Registers the function** as something the LLM can call
2. **Extracts the signature** -- the LLM knows it needs a `ticker` string and gets back a float
3. **Uses the docstring** as instructions for the LLM

**The docstring is critical.** It's not just documentation for humans -- it's how the LLM knows *when* and *how* to use the tool. A vague docstring means the LLM won't know when to reach for the tool. A clear one means it will call the right tool at the right time.

In this API version, our tools call `yfinance` under the hood, so every number is live market data.

In [5]:
@tool
def get_stock_price(ticker: str) -> float:
    """
    Get the current price for a stock ticker.

    Args:
        ticker: The stock symbol (e.g., 'AAPL', 'NVDA')

    Returns:
        The current stock price as a float
    """
    import yfinance as yf
    stock = yf.Ticker(ticker)
    return stock.info.get('regularMarketPrice', 0.0)

In [6]:
print(f"AAPL price: ${get_stock_price('AAPL')}")

AAPL price: $263.88


## Creating the Agent

`CodeAgent` combines an LLM with a set of tools. When you ask a question, the LLM decides which tools to call based on the query and the tool docstrings. Three lines is all it takes:

1. Import the components
2. Create the model
3. Create the agent with tools

The agent handles the entire loop: understanding the question, selecting the right tool, calling it, and synthesizing the response.

In [7]:
agent = CodeAgent(
    tools=[get_stock_price],
    model=model
)

print("Agent created with 1 tool: get_stock_price")

Agent created with 1 tool: get_stock_price


In [8]:
result = agent.run("What is Apple's current stock price?")
print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is Apple's current stock price?                                                                            │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  apple_stock_price = get_stock_price(ticker="AAPL")                                                               
  print(f"Apple's current stock price is: {apple_stock_price}")                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Apple's current stock price is: 263.88

Out: None

[Step 1: Duration 4.07 seconds| Input tokens: 2,022 | Output tokens: 82]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(263.88)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: 263.88

[Step 2: Duration 1.11 seconds| Input tokens: 4,224 | Output tokens: 132]

263.88


## Seeing Inside the Agent

Agents shouldn't be black boxes. Setting `verbosity_level=LogLevel.INFO` shows the agent's reasoning loop:

- What the agent **thought**
- What tool it **decided to call**
- What arguments it **passed**
- What result **came back**
- How it **synthesized** the final response

This transparency matters for debugging, compliance, and building trust. Every step is visible.

In [9]:
agent_verbose = CodeAgent(
    tools=[get_stock_price],
    model=model,
    verbosity_level=LogLevel.INFO
)

result = agent_verbose.run("What is Apple's current stock price?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is Apple's current stock price?                                                                            │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  apple_stock_price = get_stock_price(ticker='AAPL')                                                               
  final_answer(apple_stock_price)                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: 263.88

[Step 1: Duration 1.87 seconds| Input tokens: 2,022 | Output tokens: 67]

## Adding More Tools

One tool is useful. Multiple tools are powerful. The LLM **autonomously decides** which tools to use based on the question. You don't write if-else routing logic -- the agent handles it.

Three principles make tools work well:

1. **Clear docstrings** -- Tell the LLM *when* to use the tool
2. **Type hints matter** -- `ticker: str` and `-> float` tell the LLM what to pass and expect
3. **One tool, one job** -- Don't make a tool that does five things. Make five tools that each do one thing well. The agent combines them.

In [10]:
@tool
def get_company_info(ticker: str) -> str:
    """
    Get basic company information for a stock ticker.

    Args:
        ticker: The stock symbol (e.g., 'AAPL', 'NVDA')

    Returns:
        A string with company name, sector, and market cap
    """
    import yfinance as yf
    stock = yf.Ticker(ticker)
    info = stock.info
    name = info.get('longName', 'Unknown')
    sector = info.get('sector', 'Unknown')
    market_cap = info.get('marketCap', 0)
    if market_cap >= 1e12:
        mc_str = f"${market_cap/1e12:.1f}T market cap"
    elif market_cap >= 1e9:
        mc_str = f"${market_cap/1e9:.1f}B market cap"
    else:
        mc_str = f"${market_cap:,.0f} market cap"
    return f"{name} | {sector} | {mc_str}"

In [11]:
@tool
def calculate_position_value(ticker: str, shares: int) -> float:
    """
    Calculate the total value of a stock position.

    Args:
        ticker: The stock symbol
        shares: Number of shares owned

    Returns:
        Total position value in dollars
    """
    price = get_stock_price(ticker)
    return price * shares

In [12]:
agent = CodeAgent(
    tools=[get_stock_price, get_company_info, calculate_position_value],
    model=model,
    verbosity_level=LogLevel.INFO
)

print("Agent now has 3 tools!")

Agent now has 3 tools!


In [13]:
result = agent.run("Tell me about NVIDIA and what 100 shares would be worth")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Tell me about NVIDIA and what 100 shares would be worth                                                         │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  nvidia_info = get_company_info(ticker="NVDA")                                                                    
  print(nvidia_info)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
NVIDIA Corporation | Technology | $4.5T market cap

Out: None

[Step 1: Duration 3.18 seconds| Input tokens: 2,120 | Output tokens: 101]

...

[Step 3: Duration 2.07 seconds| Input tokens: 6,917 | Output tokens: 262]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({                                                                                                   
      "company_info": "NVIDIA Corporation | Technology | $4.5T market cap",                                        
      "shares_value": 18497.0                                                                                      
  })                                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: {'company_info': 'NVIDIA Corporation | Technology | $4.5T market cap', 'shares_value': 18497.0}

[Step 4: Duration 2.97 seconds| Input tokens: 9,582 | Output tokens: 351]

## Using Memory for Conversations

By default, each `agent.run()` call **resets memory** and starts fresh. To maintain context across multiple queries, use `reset=False`:

- **Without `reset=False`:** Each query is independent (no conversation)
- **With `reset=False`:** The agent remembers previous context (true conversation)

Without memory, the agent wouldn't know that "And what about NVIDIA?" refers to stock prices. With it, the agent understands the follow-up because it remembers the previous exchange.

In [14]:
# First query - starts fresh
result = agent.run("What's Apple's stock price?")
print(result)

# Follow-up - preserves memory from previous run
result = agent.run("And what about NVIDIA?", reset=False)
print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What's Apple's stock price?                                                                                     │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  apple_stock_price = get_stock_price(ticker='AAPL')                                                               
  print(f"Apple's stock price is: {apple_stock_price}")                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Apple's stock price is: 263.88

Out: None

[Step 1: Duration 2.60 seconds| Input tokens: 2,113 | Output tokens: 75]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(184.97)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: 184.97

[Step 4: Duration 1.56 seconds| Input tokens: 9,383 | Output tokens: 244]

184.97


## Inspecting Memory

You can access the agent's memory programmatically via `agent.memory.steps`. Every interaction is stored: what was asked (`TaskStep`), what tools were called (`ActionStep`), and what came back.

This is your **audit trail**. In finance, being able to show exactly what an agent did, why, and with what data is not optional -- it's a requirement.

In [15]:
print("Agent Memory:")
for i, step in enumerate(agent.memory.steps):
    step_type = type(step).__name__
    print(f"\nStep {i+1}: {step_type}")

    if step_type == "TaskStep":
        print(f"   Task: {step.task}")

    elif step_type == "ActionStep":
        if hasattr(step, 'tool_calls') and step.tool_calls:
            for tc in step.tool_calls:
                print(f"   Tool: {tc.name}({tc.arguments})")

Agent Memory:

Step 1: TaskStep
   Task: What's Apple's stock price?

Step 2: ActionStep
   Tool: python_interpreter(apple_stock_price = get_stock_price(ticker='AAPL')
print(f"Apple's stock price is: {apple_stock_price}"))

Step 3: ActionStep
   Tool: python_interpreter(final_answer(263.88))

Step 4: TaskStep
   Task: And what about NVIDIA?

Step 5: ActionStep
   Tool: python_interpreter(nvidia_stock_price = get_stock_price(ticker='NVDA')
print(f"NVIDIA's stock price is: {nvidia_stock_price}"))

Step 6: ActionStep
   Tool: python_interpreter(final_answer(184.97))


## Exercise: Build Your Own Tool

Create a `get_52_week_high` tool that returns the 52-week high price for a stock.

**Requirements:**
- Use `yfinance` to fetch live data
- Write a clear docstring that tells the LLM *when* to use this tool (highs, peaks, maximum prices)
- Include proper type hints
- Return a float

**Hint:** The yfinance `info` dictionary has a key called `fiftyTwoWeekHigh`.

In [ ]:
@tool
def get_52_week_high(ticker: str) -> float:
    """
    # YOUR DOCSTRING HERE
    # Describe what the tool does, its args, and return value
    """
    # YOUR CODE HERE
    # Use yfinance to get the 52-week high
    pass

In [ ]:
print(f"AAPL 52-week high: ${get_52_week_high('AAPL')}")

In [ ]:
agent_with_high = CodeAgent(
    tools=[get_stock_price, get_company_info, get_52_week_high],
    model=model,
    verbosity_level=LogLevel.INFO
)

result = agent_with_high.run("How far is Apple from its 52-week high?")

## Solution

Don't peek until you've tried it!

In [ ]:
@tool
def get_52_week_high(ticker: str) -> float:
    """
    Get the 52-week high price for a stock ticker.

    Use this when the user asks about highs, peaks, or
    maximum prices over the past year.

    Args:
        ticker: The stock symbol (e.g., 'AAPL', 'NVDA')

    Returns:
        The 52-week high price as a float
    """
    import yfinance as yf
    stock = yf.Ticker(ticker)
    return stock.info.get('fiftyTwoWeekHigh', 0.0)